#1. Busines Understanding

Bank X adalah salah satu bank yang memiliki basis pelanggan yang terus berkembang. Mayoritas nasabah tersebut merupakan nasabah deposan dengan ukuran simpanan yang bervariasi. Jumlah pelanggan yang juga peminjam (nasabah aset) cukup kecil, dan bank tertarik untuk memperluas basis ini dengan cepat untuk mendatangkan lebih banyak bisnis pinjaman dan dalam prosesnya, menghasilkan lebih banyak melalui bunga pinjaman. Secara khusus, manajemen ingin mencari cara untuk mengubah pelanggan pertanggungjawabannya menjadi pelanggan pinjaman pribadi (sambil mempertahankan mereka sebagai deposan).

Kampanye yang dijalankan bank tahun lalu untuk pelanggan deposan menunjukkan tingkat konversi yang sehat dengan keberhasilan lebih dari 9%. Hal ini mendorong departemen pemasaran ritel untuk merancang kampanye dengan target pemasaran yang lebih baik untuk meningkatkan rasio keberhasilan.



##2. Problem of Business

Sebagai Data Scientist di bank X, kita  harus membuat model yang akan membantu departemen pemasaran mengidentifikasi *calon pelanggan yang memiliki kemungkinan lebih tinggi untuk membeli pinjaman.*
Maka tim data scientist akan mencoba membagi menajadi problem sebagai berikut ini:
Untuk memprediksi apakah pelanggan liabilitas akan membeli pinjaman pribadi atau tidak.
   

* Menentukan apakah customer akan menerima penawaran pinjaman atau tidak    
* Variabel mana yang paling signifikan.
*  Segmen pelanggan mana yang harus lebih dibidik.
*   Apakah Umur berpengaruh terhadap pembelian pinjaman nasabah?
*   Apakah orang dengan pendapatan lebih rendah meminjam pinjaman?


##Dataset

File Bank_X.xls berisi data 5000 pelanggan. Data tersebut meliputi informasi demografis nasabah (usia, pendapatan, dll), hubungan nasabah dengan bank dan tanggapan nasabah terhadap kampanye pinjaman pribadi terakhir.Hanya 480 (= 9,6%) yang menerima pinjaman pribadi yang ditawarkan kepada mereka pada kampanye sebelumnya. Berikut ini adalah keterangan dari masing-masing kolom yang digunkanan:





    ID: ID dari customer
    Age: Umur Customer
    Experience: Tahun dari profesional experience
    Income: Annual income dari customer dalam satuan USD
    ZIP Code: Kode Pos
    Family: Size of Family from customer
    CCAvg:Rata-rata pengeluaran dari credit card(in thousand dollars)
    Education: Tingkat Pendidikan. 1: Undergrad; 2: Graduate;3: Advanced/Professional
    Mortgage: Value of house mortgage if any. (in thousand dollars)
    Personal_Loan: Status personal customer saat ditawarkan pinjama di campign terakhir
    Securities_Account: Apakah customer securities account di bank ini?
    CD_Account: ApaKAH Customer memiliki seritifkat deposit (CD) account?
    Online: Apakah customer memiliki internet banking?
    CreditCard: Apakah customer menggunakan kartu kredit dari bank lain?




Maka dari keterangan data tersebut, dan business understanding serta problem maka step-step yang kami lakukan adalah sebagai berikut ini:

#Data Preparation and Data Understanding

In [ ]:
#Melakukan penginstallan library yang dibutuhkan
### Install and IMPORT: ------------------------------------


!pip install zipcodes # installing Zipcodes library .


import scipy.stats as stats 
import pandas as pd
import numpy as np
import zipcodes as zcode # to get zipcodes

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings

import statsmodels.api as sm
#--Sklearn library--
# Sklearn package's randomized data splitting function
from sklearn.model_selection import train_test_split 

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import metrics
#AUC ROC curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay #to plot confusion matric
from sklearn.metrics import plot_confusion_matrix

from sklearn.linear_model import LogisticRegression #to build the model
from sklearn.tree import DecisionTreeClassifier#to build the model

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth',400)
pd.set_option('display.float_format', lambda x: '%.5f' % x) 
# To supress numerical display in scientific notations
warnings.filterwarnings('ignore') # To supress warnings
 # set the background for the graphs
plt.style.use('ggplot')

In [ ]:
#Melakukan pembacaan dataset dari drive

df= pd.read_csv('Bank_X.xls')
df

Dari dataframe tersebut/df maka dapat kita lihat kita sudah memasukkan menjadi dataframe dengan shape baris, kolom = 5000,14

In [ ]:
#Melakukan cek data teratas
df.head()

In [ ]:
#Melakukan cek data terbawah
df.tail()

In [ ]:
#Melakukan pengecekan size dari dataframe
print ("Rows     : " , df.shape[0])  #Jumlah Baris
print ("Columns  : " , df.shape[1]) #Jumlah Kolom
print ("#"*40,"\n","Features : \n\n", df.columns.tolist()) #Melihat nama features
print ("#"*40,"\nMissing values :\n\n", df.isnull().sum().sort_values(ascending=False)) #melihat data null
print( "#"*40,"\nPercent of missing :\n\n", round(df.isna().sum() / df.isna().count() * 100, 2)) # Menjumlahkan jika ada nila Na/null
print ("#"*40,"\nUnique values :  \n\n", df.nunique())  # Melakukan count nilai unik

Dari data tersebut dapat kita validadi bahwa jumlah customer memang benar sebanyak 5000 customer, hal ini dibuktikan nilai uniknya, dan variasi umur ada sekitar 45 variasi, dan lainnya

 Semua kolom adalah data numerical, Kolom Personal Loan adalah target yang akan kita prediksi sementara Zipcode,Family,Education,Securities Account,CD_account,online,Credit card adalah variabel yang berpengaruh terhadap variabel yang akan kita prediksi.

In [ ]:
#Mari kita melakuka sample terhadap 10 data pertama
df.head(10)

Dari dataset diatas, sepertinya index tidak kita butuhkan, maka kita bisa menghapusnya.

In [ ]:


df.drop(['ID'],axis=1,inplace=True) #Menghapus ID



Lalu dari dataset yang ada kita bisa melakukan rename dari variabel yang mengandung spasi sehingga memudahkan dalam mengolahnya.

In [ ]:
df.rename(columns={"ZIP Code":"ZIPCode","Personal Loan":"PersonalLoan","Securities Account":"SecuritiesAccount","CD Account":'CDAccount'},inplace=True)

#Data Preprosesing

###Prosesing ZipCode

Kode pos adalah fitur kategorikal dan dapat menjadi prediktor variabel target yang baik. Kita menganalisis apakah ada pola di lokasi untuk pelanggan yang telah meminjam selama kampanye sebelumnya. Jika memang tidak mempengaruhi maka kita dapat mengurangi variabel prediktornya

In [ ]:
df.ZIPCode.nunique() #Untuk melihat unik ZIPcode

Dari data tersebut dapat kita lihat bahwa dari 5000 customer, ZIPCode unik adalah 467 maka dapat kita asumsikan ada beberapa customer yang memiliki ZipCode yang sama

In [ ]:
# Untuk mendapatkan data zipcode yang unik
zipcode=df.ZIPCode.unique()

In [ ]:
#Dari zipcode diatas kita bisa melihat daerah tersebut dengan melakukan matching dengan library yang sudah kita install diatas

#here i am creating a dictionary of county by using library zipcode and matching method.
dict_zip={}
for zipcode in zipcode:
    my_city_county = zcode.matching(zipcode.astype('str'))
    if len(my_city_county)==1: # if  zipcode is present then get county else, assign zipcode to county
        county=my_city_county[0].get('county')
    else:
        county=zipcode
    
    dict_zip.update({zipcode:county})

In [ ]:
 dict_zip

Dari library yang digunakan dengan function yang kita setting sudah terdapat daerah berdasarkan zipcode, namun ada bebrapa zipcode yang tidak terdeksi yaitu zipcode 96651,92634,93077,92717. Maka bisa dilakukan pencarian di goole dengan melakukan replace pada hasil yang sudah kita buat dengan replace functio

In [ ]:


dict_zip.update({92717:'Panyula County'})
dict_zip.update({92634:'Fullerton County'})
dict_zip.update({96651:'Spain County'})
dict_zip.update({93077:'California County'})


In [ ]:
dict_zip

semua zipcode sudah terisi, maka kita bisa memuat data dari zipcode tadi menjadi kolom baru dari dataset kita dengan metode berikut ini:

In [ ]:


df['County']=df['ZIPCode'].map(dict_zip)



In [ ]:
df #melihat dataframe yang sudah kita add

In [ ]:
df.County.nunique() #melihat uniq daerah yang sudah kita hasilkan
df.info()




###Memperbaiki Tipe Data

Personal_Loan, Securities_Account, CD_Account, 'Online', 'CreditCard' ,Education memiliki tipe data int/object type, maka kita dapat mengganti menjadi object

In [ ]:


# Konversi ke tipe data katgorical
newtype = ['PersonalLoan', 'SecuritiesAccount','Family', 'CDAccount', 'Online', 'CreditCard', 'ZIPCode', 'Education','County']
df[newtype] = df[newtype].astype('category')
    



In [ ]:


df.info()



###Prosesing Experience

In [ ]:
df['Experience'].unique()

Dari data diatas kita bisa melihat bahwa ada nilai experience minus, sehingga kita coba akan melakukan experience minus itu secara average di umur berapa

In [ ]:
# checking negative and zero values for experience. 
df[df['Experience']<0]['Age'].describe()

Dari data diatas dapat kita simpulkan bahwa dari 5000 customer, terdapat 52 customer dengan grup umur 23 sampai 29 tahun punya negatif value experience, berikut ini data customer tersebut


In [ ]:
df[df['Experience']<0].sort_values(by='Experience',ascending=True)

In [ ]:
df.groupby(['Age','Education'])['Experience'].describe().T

dari data diatas bisa kita lihat dimulai umur 23 memang experience dari customer memiliki value negatif dibanding umur 40 an.

In [ ]:
#Sekarang coba kita lihat dengan 0 experince
df[df['Experience']==0]['Age'].describe()

Range usia juga terpait di 24 dan 30

#Eksploatory Data Analysis/Descriptive Statistik

In [ ]:

df.describe().T

Rata rata customer berumur di 45 tahun dengan nilai experience 20, serta Income 70 USD, maka selanjutnya kita coba akan menghitung unique dari value kita

In [ ]:
for column in newtype:
    print(df[column].value_counts())
    print("!" * 40)

Insight:

    1. Usia pelanggan berkisar antara 23 - 67, dengan rata-rata dan median ~45.
    2. Pengalaman maksimal adalah 43 tahun. di mana rata-rata dan median adalah ~20.
    3. Penghasilan berada dalam kisaran 8k hingga 224k USD. Rata-rata adalah 73k USD dan median adalah 64k USD. 224 Max gaji perlu diverifikasi
    4.Pengeluaran rata-rata untuk kartu kredit per bulan berkisar antara 1- 10k dengan rata-rata 1,9kUSD dan median 1,5k USD
    5. 1095 customer berasal dari Los Angeles County.
    6. 480 pelanggan telah meminjam pinjaman sebelumnya.
    7. >50% Customer sudah menggunakan mobile banking
    8. 70% Customer tidak menggunakan credi card
    9. dll





###Analisis tiap variabel/Unvariate Analysis

In [ ]:
def dist_box(data):
 #Melakukan pembuata boxplot, dan grafik lain denagn bebera kriteria 
    Name=data.name.upper()
    fig,(ax_box,ax_dis)  =plt.subplots(nrows=2,sharex=True,gridspec_kw = {"height_ratios": (.25, .75)},figsize=(8, 5))
    mean=data.mean()
    median=data.median()
    mode=data.mode().tolist()[0]
    sns.set_theme(style="white")
    fig.suptitle("Persebaran dari Data : "+ Name  , fontsize=30, fontweight='bold')
    sns.boxplot(x=data,showmeans=True, orient='h',color="violet",ax=ax_box)
    ax_box.set(xlabel='')
     # setting backround putih
    sns.despine(top=True,right=True,left=True) # menghapus garis samping graph
    sns.distplot(data,kde=False,color='blue',ax=ax_dis)
    ax_dis.axvline(mean, color='r', linestyle='--',linewidth=2)
    ax_dis.axvline(median, color='g', linestyle='-',linewidth=2)
    ax_dis.axvline(mode, color='y', linestyle='-',linewidth=2)
    plt.legend({'Mean':mean,'Median':median,'Mode':mode})
                    

In [ ]:
#kolom yang akan divisualisasikan
list_col=  ['Age','Experience','Income','CCAvg','Mortgage']
for i in range(len(list_col)):
    dist_box(df[list_col[i]])

Insight

    1. Usia dan pengalaman keduanya memiliki Hubungan yang sama
    2. Pendapatan condong ke kanan dan memiliki beberapa outlier di sisi yang lebih tinggi yang dapat dipotong.
    3. Kredit bulanan rata-rata miring ke kanan dan memiliki banyak 

Dari kesimpulan diatas kita bisa lihat usia merupakan salah satu pola yang mempengaruhi dari peminjama, maka di sini akan saya lakukan pengelompokan bin usia

####Usia

In [ ]:
df['Agebin'] = pd.cut(df['Age'], bins = [0, 30, 40, 50, 60, 100], labels = ['18-30', '31-40', '41-50', '51-60', '60-100'])

####Income

Income juga perlu kita bagi berdasarkan dari user seperti berikut ini:

In [ ]:




df["Income_group"] = pd.cut(
    x=df["Income"],
    bins=[0, 50, 140, 224],
    labels=["Lower", "Middle", "High"],
)



####Spending
Kita juga bisa melakukan grouping spending dari description statistikanya seperti berikut ini:

In [ ]:
df.CCAvg.describe()

In [ ]:


df["Spending_group"] = pd.cut( x=df["CCAvg"], bins=[0.00000, 0.70000, 2.50000, 10.00000],
    labels=["Low", "Medium", "High"],include_lowest=True ,
)



setelah melakukan grouping dari berbagai variabel tadi, maka kita sebaiknya melakuakn visualisasi untuk melihat insigt datanya

####Visualisasi Data in EDA

In [ ]:
# Making a list of all categorical variables
cat_columns = ['Family','Education','PersonalLoan','SecuritiesAccount',
               'CDAccount','Online','CreditCard','Agebin','Income_group','Spending_group']
title=['Jenis Family','Tingkat Pendidikan','Customer ACC Loan',
       ' Customer yang memiliki Securities Account','Customers yang memiliki a CD Account',
       'Customers  yang transaction Online',' Customers yang memiliki  Credit Card','Grup Umur',"Grup Pendapatan",'Grup Amount Yang Ditransaksikan']
plt.figure(figsize=(14,20))

sns.set_theme(style="white") # just trying to make visualisation better. This will set background to white
#list_palette=['Blues_r','Greens_r','Purples_r','Reds_r','Blues_r','Greens_r','Purples_r','Reds_r','Blues_r']

for i, variable in enumerate(cat_columns):
                     plt.subplot(5,2,i+1)
                     order = df[variable].value_counts(ascending=False).index   
                     #sns.set_palette(list_palette[i]) # to set the palette
                     sns.set_palette('Set2')
                     ax=sns.countplot(x=df[variable], data=df )
                     sns.despine(top=True,right=True,left=True) # to remove side line from graph
                     for p in ax.patches:
                           percentage = '{:.1f}%'.format(100 * p.get_height()/len(df[variable]))
                           x = p.get_x() + p.get_width() / 2 - 0.05
                           y = p.get_y() + p.get_height()
                           plt.annotate(percentage, (x, y),ha='center')
                     plt.tight_layout()
                     plt.title(title[i].upper())
                                     

Insight:

    1. 29,4% pelanggan lajang.
    2. 41,9% pelanggan adalah mahasiswa.
    3. 29,4% pelanggan memiliki kartu kredit.
    4. 10,4% nasabah memiliki rekening efek di bank
    5. 6% pelanggan memiliki akun CD.
    6. 60% pelanggan bertransaksi online.
    7. 75% pelanggan berada di kisaran 31-60.
    8.  50 % Sebagian besar nasabah bank termasuk golongan menengah.
    9. 48% pelanggan memiliki pengeluaran rata-rata sedang

####County

Melihat daerah mana yang memiliki customer yang melakukan peminjama

In [ ]:
plt.figure(figsize=(15,24))

pd.crosstab(index=df['County'],columns=df['PersonalLoan'].sort_values(ascending=False)).plot(kind='barh',stacked=True,figsize=(15,24))

Dari data diatas, berikut ini top 3 daerah yang melakukan pinjaman di bank X:
1. Los Angels
2. Sandiego
3. Santa Clara

###Biavriate/Multivariate Analysis

In [ ]:
sns.set_palette(sns.color_palette("Set2", 8))
plt.figure(figsize=(15,10))
sns.heatmap(df.corr(),annot=True)
plt.show()

Insight:
1.  Usia dan pengalaman sangat berkorelasi 
2. Pendapatan dan pengeluaran rata-rata pada kartu kredit berkorelasi positif.

In [ ]:

#Visualisasi Spending dengan Status Loan
sns.relplot(x='Income_group',y='CCAvg',hue='PersonalLoan',data=df)
sns.despine(top=True,right=True,left=True) # to remove side line from graph



Insight : Ketika Spending tinggi, maka rata rata customer akan melakukan loan juga

####Age and Loan

In [ ]:


sns.relplot(x='Age',y='CCAvg',hue='PersonalLoan',data=df)
sns.despine(top=True,right=True,left=True) # to remove side line from graph



####Family and loan

In [ ]:


sns.relplot(x='Family',y='CCAvg',hue='PersonalLoan',data=df)
sns.despine(top=True,right=True,left=True) # to remove side line from graph



Loan tertinggi berada di umur 40 an

###Eduactaion and Status Loan

In [ ]:
sns.swarmplot(x='Education',y='Income',hue='PersonalLoan',data=df)
sns.despine(top=True,right=True,left=True) # to remove side line from graph
labels=["No","Yes"]
plt.legend(loc='lower left', frameon=False,)
plt.legend(loc="upper left", labels=labels,title="Borrowed Loan",bbox_to_anchor=(1,1))

Dari sini kita bisa melihat education tingkat 2 dan 3 melakukan loan

###Insight of EDA

1. Variabel dependen adalah Personal_loan yang bertipe data kategorikal.
Age, Experience, Income,mortage ,CCavg bertipe integer sedangkan variabel lainnya bertipe kategoris
3. Tidak ada nilai yang hilang dalam kumpulan data.
4. Orang dengan pendapatan lebih tinggi telah memilih pinjaman pribadi sebelumnya.
5. Jika pelanggan memiliki lebih tinggi rata-rata penggunaan kredit bulanan akan memilih pinjaman.
6. Pelanggan dengan Keluarga dari 3/lebih anggota telah meminjam lebih banyak pinjaman dengan bank.
7. Tingkat pendidikan 2: Sarjana dan 3: Lanjutan/Professional pernah meminjam ke bank.
8. Rasio pinjaman pinjaman tinggi di 30 dan di bawah dan 60 dan di atas pelanggan.
9. Semakin banyak pendapatan, semakin banyak membelanjakan dan memiliki gaya hidup "besar dari kehidupan".

Segmentasi nasabah untuk meminjam pinjaman berdasarkan EDA

1. Pelanggan dengan pendapatan lebih tinggi dan pengeluaran rata-rata bulanan yang lebih tinggi. Mereka juga memiliki sertifikat deposito dengan bank. Mereka adalah top customer.
2. Beberapa Pelanggan dalam kelompok berpenghasilan menengah, memiliki pengeluaran kartu kredit bulanan yang lebih sedikit. Mereka adalah customer profil rata-rata.
3. Pelanggan dalam kelompok berpenghasilan rendah, lebih sedikit pengeluaran bulanan. Mereka adalah customer low profile.

Tindakan untuk pra-pemrosesan data:

1. Banyak variabel memiliki outlier yang perlu diperlakukan.


####Outliers Detection

In [ ]:
numeric_columns =['Income','CCAvg','Mortgage','Age']
# outlier detection using boxplot
plt.figure(figsize=(20,30))

for i, variable in enumerate(numeric_columns):
                     plt.subplot(4,4,i+1)
                     plt.boxplot(df[variable],whis=1.5)
                     plt.tight_layout()
                     plt.title(variable)

plt.show()

In [ ]:
# Check Income extreme values
df.sort_values(by=["Income"],ascending = False).head(5)

Menambahkan categori regions

In [ ]:
counties = {
'Los Angeles County':'Los Angeles Region',
'San Diego County':'Southern',
'Santa Clara County':'Bay Area',
'Alameda County':'Bay Area',
'Orange County':'Southern',
'San Francisco County':'Bay Area',
'San Mateo County':'Bay Area',
'Sacramento County':'Central',
'Santa Barbara County':'Southern',
'Yolo County':'Central',
'Monterey County':'Bay Area',            
'Ventura County':'Southern',             
'San Bernardino County':'Southern',       
'Contra Costa County':'Bay Area',        
'Santa Cruz County':'Bay Area',           
'Riverside County':'Southern',            
'Kern County':'Southern',                 
'Marin County':'Bay Area',                
'San Luis Obispo County':'Southern',     
'Solano County':'Bay Area',              
'Humboldt County':'Superior',            
'Sonoma County':'Bay Area',                
'Fresno County':'Central',               
'Placer County':'Central',                
'Butte County':'Superior',               
'Shasta County':'Superior',                
'El Dorado County':'Central',             
'Stanislaus County':'Central',            
'San Benito County':'Bay Area',          
'San Joaquin County':'Central',           
'Mendocino County':'Superior',             
'Tuolumne County':'Central',                
'Siskiyou County':'Superior',              
'Trinity County':'Superior',                
'Merced County':'Central',                  
'Lake County':'Superior',                 
'Napa County':'Bay Area',                   
'Imperial County':'Southern',
93077:'Southern',
96651:'Bay Area'
}


df['Regions'] = df['County'].map(counties)



Ini adalah beberapa outlier, seperti pendapatan 224K USD.  Setelah mengidentifikasi outlier, maka dapat kita putuskan apakah akan menghapus/memperlakukannya atau tidak. Karena setelah diskusi dengan user itu bisa digunakan maka kita menggunakan untuk melihat potential customer untuk melakukan peminjaman

###Melakukan penghapusan kolom yang tidak kita gunakan

In [ ]:

df.drop(columns=["Agebin", "ZIPCode","County",'Experience','Income_group','Spending_group'], inplace=True)

In [ ]:

X = df.drop(['PersonalLoan'], axis=1)
Y = df['PersonalLoan']

oneHotCols=['Regions','Education']
X=pd.get_dummies(X,columns=oneHotCols,drop_first=True)



###Split data to Train and Test secara acak

In [ ]:
#Splitting data in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.30, random_state = 1,stratify=Y)

###Melakukan Scaling Data

In [ ]:
from sklearn.preprocessing import StandardScaler
# Creating StandardScaler instance
scaler = StandardScaler()

# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled,columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled,columns=X_test.columns)


X_train_scaled_df.index=np.arange(len(X_train_scaled_df))
X_test_scaled_df.index=np.arange(len(X_test_scaled_df))
y_train.index=np.arange(len(y_train))
y_test.index=np.arange(len(y_test))

#Membangun Model Logistic Regresion

Asmsi model akan menghasilkan:

Kasus 1. Memprediksi seseorang akan membeli pinjaman tetapi sebenarnya tidak. 
Kasus 2. Memprediksi seseorang tidak akan membeli pinjaman tetapi actual dia melakukan

Kasus mana yang lebih penting?

Seluruh tujuan kampanye adalah untuk mendatangkan lebih banyak pelanggan. Kasus ke-2 lebih penting bagi bank. Sehingga pelanggan potensial terlewatkan oleh tim penjualan/pemasaran. 

Bagaimana cara mengurangi kerugian/peluang tadi ? 
yaitu perlu mengurangi False Negatif

    Dalam hal ini, tidak dapat mengidentifikasi pelanggan potensial adalah kerugian terbesar yang dapat di hadapi bank. Oleh karena itu, penarikan kembali adalah metrik yang tepat untuk memeriksa kinerja model. Bank ingin Inga dimaksimalkan, semakin besar recall, semakin kecil kemungkinan False Negatif(predict negatif, but actual True => False Negatif).

Kita dapat menggunakan akurasi, tetapi karena datanya imbalance, ini bukan metrik yang tepat untuk memeriksa kinerja model.

    Oleh karena itu, Recall harus dimaksimalkan, semakin besar Recall semakin tinggi peluang mengidentifikasi kedua kelas dengan benar. Maka unsur yang dapat mdigunakan dalam mengevaluasi model di segemntasi customer banking adalah recall

In [ ]:
def make_confusion_matrix(y_actual,y_predict,title):
    fig, ax = plt.subplots(1, 1)
    
    cm = confusion_matrix(y_actual, y_predict, labels=[0,1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=["No","Yes"])
    disp.plot(cmap='Greens',colorbar=True,ax=ax)
    ax.set_title(title)
    plt.tick_params(axis=u'both', which=u'both',length=0)
    plt.grid(b=None,axis='both',which='both',visible=False)
    plt.show()

In [ ]:
def get_metrics_score(model,X_train_df,X_test_df,y_train_pass,y_test_pass,statsklearn,threshold=0.5,flag=True,roc=False):
    '''
    Function to calculate different metric scores of the model - Accuracy, Recall, Precision, and F1 score
    model: classifier to predict values of X
    X_train_df, X_test_df: Independent features
    y_train_pass,y_test_pass: Dependent variable
    statsklearn : 0 if calling for Sklearn model else 1
    threshold: thresold for classifiying the observation as 1
    flag: If the flag is set to True then only the print statements showing different will be displayed. The default value is set to True.
    roc: If the roc is set to True then only roc score will be displayed. The default value is set to False.
    '''
    # defining an empty list to store train and test results
    
    score_list=[] 
    if statsklearn==0:
        pred_train = model.predict(X_train_df)
        pred_test = model.predict(X_test_df)
    else:
        pred_train = (model.predict(X_train_df)>threshold)
        pred_test = (model.predict(X_test_df)>threshold)
    
    
    pred_train = np.round(pred_train)
    pred_test = np.round(pred_test)
    
    train_acc = accuracy_score(y_train_pass,pred_train)
    test_acc = accuracy_score(y_test_pass,pred_test)
    
    train_recall = recall_score(y_train_pass,pred_train)
    test_recall = recall_score(y_test_pass,pred_test)
    
    train_precision = precision_score(y_train_pass,pred_train)
    test_precision = precision_score(y_test_pass,pred_test)
    
    train_f1 = f1_score(y_train_pass,pred_train)
    test_f1 = f1_score(y_test_pass,pred_test)
    
    
    score_list.extend((train_acc,test_acc,train_recall,test_recall,train_precision,test_precision,train_f1,test_f1))
      
    if flag == True: 
        print("\x1b[0;30;47m \033[1mMODEL PERFORMANCE\x1b[0m")
        print("\x1b[0;30;47m \033[1mAccuracy   : Train:\x1b[0m",
              round(accuracy_score(y_train_pass,pred_train),3),
              "\x1b[0;30;47m \033[1mTest:\x1b[0m ",
              round(accuracy_score(y_test_pass,pred_test),3))
        print("\x1b[0;30;47m \033[1mRecall     : Train:\x1b[0m"
              ,round(recall_score(y_train_pass,pred_train),3),
              "\x1b[0;30;47m \033[1mTest:\x1b[0m" ,
              round(recall_score(y_test_pass,pred_test),3))
        
        print("\x1b[0;30;47m \033[1mPrecision  : Train:\x1b[0m",
              round(precision_score(y_train_pass,pred_train),3),
              "\x1b[0;30;47m \033[1mTest:\x1b[0m ",
              round(precision_score(y_test_pass,pred_test),3))
        print("\x1b[0;30;47m \033[1mF1         : Train:\x1b[0m",
              round(f1_score(y_train_pass,pred_train),3),
              "\x1b[0;30;47m \033[1mTest:\x1b[0m",
              round(f1_score(y_test_pass,pred_test),3))
        make_confusion_matrix(y_train_pass,pred_train,"Confusion Matrix for Train")     
        make_confusion_matrix(y_test_pass,pred_test,"Confusion Matrix for Test") 
   
    if roc == True:
        
        print("\x1b[0;30;47m \033[1mROC-AUC Score  :Train:\x1b[0m: ",
              round(roc_auc_score(y_train_pass,pred_train),3),
              "\x1b[0;30;47m \033[1mTest:\x1b[0m: ",
              round(roc_auc_score(y_test_pass,pred_test),3))
    
    return score_list # returning the list with train and test scores

In [ ]:
# # defining empty lists to add train and test results
acc_train = []
acc_test = []
recall_train = []
recall_test = []
precision_train = []
precision_test = []
f1_train = []
f1_test = []

def add_score_model(score):
     '''Add scores to list so that we can compare all models score together'''   
     acc_train.append(score[0])
     acc_test.append(score[1])
     recall_train.append(score[2])
     recall_test.append(score[3])
     precision_train.append(score[4])
     precision_test.append(score[5])
     f1_train.append(score[6])
     f1_test.append(score[7])

In [ ]:
#Logistik with SKLEARN LIB
lr = LogisticRegression(solver='newton-cg',random_state=1,fit_intercept=False,class_weight={0:0.15,1:0.85})
model  = lr.fit(X_train_scaled_df,y_train)

statmodel=0  #0 for sklearn and 1 for statmodel

# Let's check model performances for this model
scores_Sklearn = get_metrics_score(model,X_train_scaled_df,X_test_scaled_df,y_train,y_test,statmodel)

###Logistik Regression with stat Model

In [ ]:
# adding constant to training and test set
X_train_stat = sm.add_constant(X_train_scaled_df)
X_test_stat = sm.add_constant(X_test_scaled_df)
statmodel=1  #0 for sklearn and 1 for statmodel
logit = sm.Logit( y_train, X_train_stat.astype(float) )
lg = logit.fit(warn_convergence=False)

# Let's check model performances for this model
scores_statmodel = get_metrics_score(lg,X_train_stat,X_test_stat,y_train,y_test,statmodel)
lg.summary() 

#Decision Tree

In [ ]:
#drop column which we don't need for modelling
df.drop(columns=["Agebin", "ZIPCode","County",'Experience','Income_group','Spending_group'], inplace=True)

In [ ]:
X_dt = df.drop('PersonalLoan', axis=1)
y_dt = df['PersonalLoan']

In [ ]:


#oneHotCols=['Regions']
oneHotCols=X_dt.select_dtypes(exclude='number').columns.to_list()
X_dt=pd.get_dummies(X_dt,columns=oneHotCols,drop_first=True)
# Spliting data set
X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(X_dt, y_dt, test_size=0.3, random_state=1, stratify=y_dt)



####Memabngun Models

In [ ]:
##  Function to calculate recall score
def get_recall_score(model):
    '''
    model : classifier to predict values of X

    '''
    ytrain_predict = model.predict(X_train_dt)
    ytest_predict = model.predict(X_test_dt)
    # accuracy on training set
    print("\x1b[0;30;47m \033[1mAccuracy : Train :\033[0m", 
          model.score(X_train_dt,y_train_dt),
          "\x1b[0;30;47m \033[1mTest:\033[0m", 
          model.score(X_test_dt,y_test_dt))
# accuracy on training set
    print("\x1b[0;30;47m \033[1mRecall   : Train :\033[0m", 
          metrics.recall_score(y_train_dt,ytrain_predict),
          "\x1b[0;30;47m \033[1mTest:\033[0m", 
          metrics.recall_score(y_test_dt,ytest_predict))
    make_confusion_matrix(y_train_dt,ytrain_predict,"Confusion Matric on Train Data")
    make_confusion_matrix(y_test_dt,ytest_predict,"Confusion Matric on Test Data")

In [ ]:
#since data is imbalanced adding weights
model = DecisionTreeClassifier(criterion = 'gini',class_weight={0:0.15,1:0.85}, random_state=1)
model.fit(X_train_dt, y_train_dt)
get_recall_score(model)

In [ ]:


column_names = list(X_dt.columns)
feature_names = column_names
print(feature_names)



In [ ]:


plt.figure(figsize=(20,30))
from sklearn import tree
from sklearn.model_selection import GridSearchCV
out = tree.plot_tree(model,feature_names=feature_names,filled=True,fontsize=9,node_ids=True,class_names=True)
for o in out:
     arrow = o.arrow_patch
     if arrow is not None:
        arrow.set_edgecolor('black')
        arrow.set_linewidth(1)
plt.show()



####Melakukan Deteksi Features Imporatnce

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='purple', align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

Terlalu banyak pohon keputusan, sehingga kita bisa melakukan tuning terhadap parameter yang berlebih ini/hyperparameter tuning denagn metode GridSearch dibawah ini:

In [ ]:
# Choose the type of classifier. 
estimator = DecisionTreeClassifier(random_state=1)

# Grid of parameters to choose from

parameters = {'max_depth': np.arange(1,10), 
              'min_samples_leaf': [1, 2, 5, 7, 10,15,20],
              'max_leaf_nodes' : [5, 10,15,20,25,30],
              }

# Type of scoring used to compare parameter combinations
acc_scorer = metrics.make_scorer(metrics.recall_score)

# Run the grid search
grid_obj = GridSearchCV(estimator, parameters, scoring=acc_scorer,cv=5)
grid_obj = grid_obj.fit(X_train_dt, y_train_dt)

# Set the clf to the best combination of parameters
estimator = grid_obj.best_estimator_
estimator

In [ ]:
# Fit the best algorithm to the data. 
estimator.fit(X_train_dt, y_train_dt)
ytrain_predict=estimator.predict(X_train_dt)
ytest_predict=estimator.predict(X_test_dt)

In [ ]:
plt.figure(figsize=(15,10))

out = tree.plot_tree(estimator,feature_names=feature_names,filled=True,fontsize=9,node_ids=False,class_names=True)
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor('black')
        arrow.set_linewidth(1)
plt.show()

In [ ]:
importances = estimator.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:


get_recall_score(estimator)



Insigt :

Dengan HyperParameter max_depth=6, max_leaf_nodes=20, min_samples_leaf=7 overfitting pada kereta telah berkurang, tetapi penarikan kembali untuk pengujian belum membaik.
  
  Fitur penting adalah Penghasilan, Eduaction,Family CCavg & Umur.
    Namun metrik penarikan masih 84 dari testing dan negatif palsu 23. Sebagagai perusahaan pastinya kita tidak ingin kehilangan peluang dalam memprediksi pelanggan ini. jadi Mari kita lihat apakah alih-alih pre pruning , post pruning membantu mengurangi false negative.

####Pruning

In [ ]:
clf = DecisionTreeClassifier(random_state=1)
path = clf.cost_complexity_pruning_path(X_train_dt, y_train_dt)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")
plt.show()



Selanjutnya, cara ini melatih pohon keputusan menggunakan alpha yang efektif dan akan menetapkan nilai alpha ini dan meneruskannya ke parameter ccp_alpha dari DecisionTreeClassifier. Kita dapat meningkatan keakuratan train and test dengan metode ini

In [ ]:
clfs = []
accuracy_train=[]
accuracy_test=[]
recall_train=[]
recall_test=[]
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=1, ccp_alpha=ccp_alpha,class_weight = {0:0.15,1:0.85})
    clf.fit(X_train_dt, y_train_dt)
    y_train_pred=clf.predict(X_train_dt)
    y_test_pred=clf.predict(X_test_dt)
    accuracy_train.append(clf.score(X_train_dt,y_train_dt))
    accuracy_test.append(clf.score(X_test_dt,y_test_dt))
    recall_train.append(metrics.recall_score(y_train_dt,y_train_pred))
    recall_test.append(metrics.recall_score(y_test_dt,y_test_pred))
    clfs.append(clf)

In [ ]:


fig, ax = plt.subplots(figsize=(10,5))
ax.set_xlabel("alpha")
ax.set_ylabel("Recall")
ax.set_title("Recall vs alpha for training and testing sets")
ax.plot(ccp_alphas, recall_train, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, recall_test, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.show()
   



Kita dapat melihat alpha sekitar 0.002 s/d 0.005 kita pilih 0.01

In [ ]:
best_model = DecisionTreeClassifier(ccp_alpha=0.002,
                       class_weight={0: 0.15, 1: 0.85}, random_state=1)
best_model.fit(X_train_dt, y_train_dt)

In [ ]:


get_recall_score(best_model)



In [ ]:


plt.figure(figsize=(15,10))

out = tree.plot_tree(best_model,feature_names=feature_names,filled=True,fontsize=9,node_ids=False,class_names=True)
for o in out:
    arrow = o.arrow_patch
    if arrow is not None:
        arrow.set_edgecolor('black')
        arrow.set_linewidth(1)
plt.show()



In [ ]:


importances = best_model.feature_importances_
indices = np.argsort(importances)

plt.figure(figsize=(12,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='violet', align='center')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()





Insight:
Dari best model kita peroleh recall bagus ketika alpha di 0.002, sehingga kita cova alpha 0.003 dan menghasilkan akurasi recall 96% dan mengurangi FN dari 21 menjadi 6 yang artinya semakin kecil resiko kehilangan potensial customer yang akan melakukan transaksi ke kita.


#Comparison semua model

In [ ]:
comparison_frame = pd.DataFrame({'Model':['scores_statmodel',
                                          'Initial decision tree model',
                                          'Decision treee with hyperparameter tuning',
                                          'Decision tree with post-pruning'], 
                                          'Train_accuracy':[0.92,1,0.99,0.98],
                                          'Test_accuracy':[0.91,0.98,0.98,0.97],
                                          'Train_Recall':[0.90,1,0.92,0.98], 
                                          'Test_Recall':[0.88,0.86,0.84,0.96]})  

comparison_frame

In [ ]:
y_pred = best_model.predict(X_test_dt)
print(classification_report(y_test_dt,y_pred))
make_confusion_matrix(y_test,y_pred,"confusion matrix on test")

Model decision tree with prooning telah memberi kami skor ingatan terbaik pada data dengan akurasi 96%. Analisis data eksplorasi juga menyarankan income and education merupakan fitur penting dalam memutuskan apakah seseorang akan meminjam pinjaman pribadi. Jadi dari semua model lebih baik menggunakan decision tree dengan alpha 0.002 atau setelah prooning untuk dataset ini.

#Kesimpulan

1. Data digunakan untuk menganalisis data promosi Pinjaman Pribadi menggunakan EDA dan dengan menggunakan model yang berbeda seperti Regresi Logistik dan Decision Tree Keputusan untuk membangun kemungkinan Pelanggan membeli Pinjaman.
2. Pertama sebaiknya membangun model menggunakan Regresi Logistik dan metrik kinerja yang digunakan adalah Recall. Fitur yang paling penting untuk klasifikasi adalah Pendapatan, Pendidikan, Keluarga dan CCAvg .
3. Koefisien Pendapatan, Lulusan dan Pendidikan, Family_3,Family 4,CCavg,CD account,Usia, adalah positif , yaitu peningkatan satu unit dalam hal ini akan meningkatkan peluang seseorang untuk meminjam pinjaman
4. Tim DS Bank X menyarankan menggunakan decision tree dengan prepruning dan post pruning. Model Post pruning memberikan 96% recall dengan akurasi 97%.
5. Penghasilan, Pelanggan dengan gelar sarjana, pelanggan yang memiliki 3 anggota keluarga adalah beberapa variabel terpenting dalam memprediksi apakah pelanggan akan membeli pinjaman pribadi.

#Recomendation And Actionable

1. Pohon keputusan tidak memerlukan banyak persiapan data atau penanganan outlier seperti regresi logistik. Mereka mudah dimengerti. Pohon keputusan dapat dengan mudah overfit(perilaku pembelajaran mesin yang tidak diinginkan yang terjadi ketika model pembelajaran mesin memberikan prediksi akurat untuk data pelatihan tetapi tidak untuk data baru/testing) , jadi kita harus berhati-hati menggunakan pohon keputusan.
2. Berdasarkan EDA, Logistic Regression , Decision tree , Income ,Educatoin,Family,CCavg merupakan faktor yang paling penting.
3. Pelanggan yang memiliki pendapatan di atas 98k dolar , Pendidikan tingkat lanjut / sarjana, keluarga lebih dari 2, pelanggan tersebut memiliki peluang lebih tinggi untuk mengambil pinjaman pribadi.
Jadi untuk kampanye ini kami dapat memiliki profil yang berbeda untuk pelanggan.
   
Action to Customer:

1. Klien Profil Tinggi:-Pendapatan lebih tinggi,Pendidikan tingkat lanjut/Sarjana, 3/4 anggota keluarga,belanja tinggi
2. Profil Rata-Rata :- Kelompok berpendapatan sedang, Pendidikan tingkat S2.3/4 Anggota keluarga, Pengeluaran sedang
3. Profil Rendah:-Kelompok berpenghasilan rendah, sarjana, 3/4 Anggota Keluarga, pengeluaran rendah

Pertama-tama, tim DS menyarankan dapat menargetkan pelanggan profil tinggi, dengan memberi mereka customer relationship yang dapat menangani masalah di sana dan dapat mengejar mereka untuk membeli pinjaman dari bank dengan suku bunga lengkap.
Target kedua adalah pelanggan profil Menengah.


Thankyou----